# Домашнее задание по теме "Многослойная нейронная сеть"

## Задание

Постройте модель на основе полносвязных слоёв для классификации Fashion MNIST из библиотеки torchvision (datasets).
Получите качество на тестовой выборке не ниже 88%

## Решение

### Импорт необходимых библиотек

In [122]:
import time

import torch
from torch.utils.data import DataLoader, random_split
import torchvision as tv
from  torchvision import transforms

### Пременные

In [ ]:
BATCH_SIZE = 256
manual_seed = 42

### Загрузка данных

In [126]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5))
])

In [127]:
full_train_dataset = tv.datasets.FashionMNIST('.', train=True, transform=transform, download=True)
test_dataset = tv.datasets.FashionMNIST('.', train=False, transform=transform, download=True)

In [128]:
full_train_dataset[0][0].shape

torch.Size([1, 28, 28])

### Разделение обучающего датасета на обучающую и валидационную выборку

In [129]:
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
print(f"tarin_size = {train_size}")
print(f"val_size = {val_size}")

tarin_size = 48000
val_size = 12000


In [130]:
train_dataset, val_dataset = random_split(
    full_train_dataset,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(manual_seed)
)

In [131]:
train = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

In [132]:
val = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [133]:
test = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

### Модель

In [134]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [226]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 512),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(512),
    torch.nn.Linear(512, 256),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(256),
    torch.nn.Linear(256, 128),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(128),
    torch.nn.Linear(128, 10)
)

model = model.to(device)
model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): Linear(in_features=512, out_features=256, bias=True)
  (5): ReLU()
  (6): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Linear(in_features=256, out_features=128, bias=True)
  (8): ReLU()
  (9): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): Linear(in_features=128, out_features=10, bias=True)
)

In [227]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=.005)
num_epochs = 15

In [184]:
def training_loop():
    for epoch in range(num_epochs):
        train_iters, train_passed = 0, 0
        train_loss, train_acc = 0., 0.
        start=time.time()

        model.train()
        for X, y in train:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_pred = model(X)
            l = loss(y_pred, y)
            l.backward()
            optimizer.step()
            train_loss += l.item()
            train_acc += (y_pred.argmax(dim=1) == y).sum().item()
            train_iters += 1
            train_passed += len(X)

        val_iters, val_passed = 0, 0
        val_loss, val_acc = 0., 0.
        model.eval()
        for X, y in val:
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            l = loss(y_pred, y)
            val_loss  += l.item()
            val_acc += (y_pred.argmax(dim=1) == y).sum().item()
            val_iters += 1
            val_passed += len(X)

        print("epoch: {}, taked: {: .3f}, train_loss: {}, train_acc: {}, val_loss: {}, val_acc: {}".format(
            epoch, time.time() - start, train_loss / train_iters, train_acc / train_passed,
            val_loss / val_iters, val_acc / val_passed
        ))   

In [228]:
training_loop()

epoch: 0, taked:  14.622, train_loss: 0.4877846153809669, train_acc: 0.8214375, val_loss: 0.42984733239133305, val_acc: 0.84725
epoch: 1, taked:  14.440, train_loss: 0.3797557198620857, train_acc: 0.8593333333333333, val_loss: 0.40489046941412254, val_acc: 0.8543333333333333
epoch: 2, taked:  13.887, train_loss: 0.3410563824024606, train_acc: 0.8738125, val_loss: 0.4199260910774799, val_acc: 0.8553333333333333
epoch: 3, taked:  13.889, train_loss: 0.32415946430348336, train_acc: 0.8789166666666667, val_loss: 0.4039122297408733, val_acc: 0.8629166666666667
epoch: 4, taked:  14.413, train_loss: 0.3008923899144568, train_acc: 0.888, val_loss: 0.37447173291064323, val_acc: 0.8726666666666667
epoch: 5, taked:  14.578, train_loss: 0.2873751166019034, train_acc: 0.8921875, val_loss: 0.38609771684129185, val_acc: 0.8726666666666667
epoch: 6, taked:  14.585, train_loss: 0.26690304826231714, train_acc: 0.899, val_loss: 0.42983080415015523, val_acc: 0.8629166666666667
epoch: 7, taked:  14.797, tr

In [230]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for X, y in test:
        X, y = X.to(device), y.to(device)
        outputs = model(X)
        _, predicted = torch.max(outputs.data,1)
        total += y.size(0)
        correct += (predicted == y).sum().item()
accuracy = 100 * correct / total
if accuracy >= 88:
    print(f"Точность модели на тестовой выборке: {accuracy:.2f}%. Поставленная цель достигнута.")
else:
    print(f"Точность модели на тестовой выборке: {accuracy:.2f}%. Поставленная цель еще не достигнута. Удачи!")    

Точность модели на тестовой выборке: 88.12%. Поставленная цель достигнута.
